In [2]:
import statsapi
import pandas as pd
import numpy as np

In [32]:
def camelcase_to_snakecase(pandas_columns):
    return pandas_columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True) \
                         .str.lower()

<h3>Schedule API</h3>

In [33]:
sched = statsapi.schedule(start_date='01/01/2020',end_date='12/31/2024')
sched_df = pd.DataFrame(sched)
sched_df.columns = camelcase_to_snakecase(sched_df.columns)
sched_df.to_csv('./test_output/schedule.csv', index=False)

In [116]:
sched_df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,...,venue_id,venue_name,national_broadcasts,series_status,summary,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher
0,607243,2020-02-21T18:05:00Z,2020-02-21,E,Completed Early,Northeastern Huskies,Boston Red Sox,343,111,N,...,4309,JetBlue Park,[],BOS wins 1-0,2020-02-21 - Northeastern Huskies @ Boston Red...,NaN,NaN,NaN,NaN,NaN
1,607244,2020-02-21T18:05:00Z,2020-02-21,E,Final,Southeastern Fire,Detroit Tigers,4284,116,N,...,2511,Publix Field at Joker Marchant Stadium,[],DET wins 1-0,2020-02-21 - Southeastern Fire (4) @ Detroit T...,Detroit Tigers,Southeastern Fire,Jared Tobey,Carlos Soto,None
2,611068,2020-02-21T20:05:00Z,2020-02-21,S,Final,Texas Rangers,Kansas City Royals,140,118,N,...,2603,Surprise Stadium,[],TEX wins Spring,2020-02-21 - Texas Rangers (5) @ Kansas City R...,Texas Rangers,Kansas City Royals,James Jones,Chance Adams,None
3,607184,2020-02-21T23:05:00Z,2020-02-21,E,Completed Early,Minnesota Golden Gophers,Minnesota Twins,4864,142,N,...,2862,CenturyLink Sports Complex,[],None,2020-02-21 - Minnesota Golden Gophers @ Minnes...,NaN,NaN,NaN,NaN,NaN
4,606181,2020-02-22T18:05:00Z,2020-02-22,S,Final,New York Mets,St. Louis Cardinals,121,138,N,...,2520,Roger Dean Chevrolet Stadium,[],STL wins Spring,2020-02-22 - New York Mets (0) @ St. Louis Car...,St. Louis Cardinals,New York Mets,Jack Flaherty,Marcus Stroman,Génesis Cabrera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370,635896,2020-10-22T00:08:00Z,2020-10-21,W,Final,Tampa Bay Rays,Los Angeles Dodgers,139,119,N,...,5325,Globe Life Field,"[FOX-INT, FOX, ESPN Radio]",Series tied 1-1,2020-10-21 - Tampa Bay Rays (6) @ Los Angeles ...,Tampa Bay Rays,Los Angeles Dodgers,Nick Anderson,Tony Gonsolin,Diego Castillo
1371,635923,2020-10-24T00:08:00Z,2020-10-23,W,Final,Los Angeles Dodgers,Tampa Bay Rays,119,139,N,...,5325,Globe Life Field,"[FOX-INT, FOX, ESPN Radio]",LAD leads 2-1,2020-10-23 - Los Angeles Dodgers (6) @ Tampa B...,Los Angeles Dodgers,Tampa Bay Rays,Walker Buehler,Charlie Morton,None
1372,635897,2020-10-25T00:08:00Z,2020-10-24,W,Final,Los Angeles Dodgers,Tampa Bay Rays,119,139,N,...,5325,Globe Life Field,"[FOX-INT, FOX, ESPN Radio]",Series tied 2-2,2020-10-24 - Los Angeles Dodgers (7) @ Tampa B...,Tampa Bay Rays,Los Angeles Dodgers,John Curtiss,Kenley Jansen,None
1373,635887,2020-10-26T00:08:00Z,2020-10-25,W,Final,Los Angeles Dodgers,Tampa Bay Rays,119,139,N,...,5325,Globe Life Field,"[FOX-INT, FOX, ESPN Radio]",LAD leads 3-2,2020-10-25 - Los Angeles Dodgers (4) @ Tampa B...,Los Angeles Dodgers,Tampa Bay Rays,Clayton Kershaw,Tyler Glasnow,Blake Treinen


<h3>Lincescore API</h3>

In [119]:
# Break count for testing
count = 0

game_linescores = []

for index, row in sched_df.iterrows():
    game_id, home_id, away_id = row['game_id'], row['home_id'], row['away_id']

    linescore = statsapi.get('game_linescore', params={'ver': 'v1', 'gamePk': game_id})

    linescore = {
        'game_id': game_id,
        'home_team_id': home_id,
        'home_team_runs': linescore['teams']['home']['runs'],
        'home_team_hits':  linescore['teams']['home']['hits'],
        'home_team_errors':  linescore['teams']['home']['errors'],
        'home_team_left_on_base':  linescore['teams']['home']['leftOnBase'],
        'home_team_is_winner': linescore['teams']['home']['isWinner'],
        'away_team_id': away_id,
        'away_team_runs': linescore['teams']['away']['runs'],
        'away_team_hits':  linescore['teams']['away']['hits'],
        'away_team_errors':  linescore['teams']['away']['errors'],
        'away_team_left_on_base':  linescore['teams']['away']['leftOnBase'],
        'away_team_is_winner': linescore['teams']['away']['isWinner'],
    }

    game_linescores.append(linescore)

    count += 1

    if count > 5:
        break

game_linescores_df = pd.DataFrame(game_linescores)

<h3>Player Batting and Pithcing Stats by Game API</h3>

In [51]:
def get_game_boxscore_stats(game_id):
    boxscore = statsapi.boxscore_data(game_id)
    batter_stats_df = get_boxscore_game_batters_stats(boxscore)
    pitcher_stats_df = get_boxscore_game_pitchers_stats(boxscore)

    return batter_stats_df, pitcher_stats_df


def get_boxscore_game_pitchers_stats(boxscore_obj):
    team_info = get_boxscore_teams(boxscore_obj)

    home_pitcher_stats_df = get_boxscore_team_pitcher_stats(boxscore_obj['homePitchers'])
    away_pitcher_stats_df = get_boxscore_team_pitcher_stats(boxscore_obj['awayPitchers'])

    home_pitcher_stats_df['team_id'] = team_info['home_team_id']
    away_pitcher_stats_df['team_id'] = team_info['away_team_id']

    if list(home_pitcher_stats_df.columns) != list(away_pitcher_stats_df.columns):
        away_pitcher_stats_df = away_pitcher_stats_df[list(home_pitcher_stats_df.columns)]

    pitcher_stats_df = pd.concat([home_pitcher_stats_df, away_pitcher_stats_df], ignore_index=True)  
    return pitcher_stats_df 


def get_boxscore_team_pitcher_stats(pitchers_list):
    pitchers_column_mapper = {
        'ip': 'innings_pitched',
        'h': 'hits',
        'r': 'runs',
        'er': 'earned_runs',
        'bb': 'walks',
        'k': 'strikeouts',
        'hr': 'home_runs',
        'era': 'earned_run_average',
        'p': 'pitches',
        's': 'strikes',
        'personId': 'person_id'
    }

    pitcher_stats_df = pd.DataFrame(pitchers_list)
    pitcher_stats_df = pitcher_stats_df.rename(columns=pitchers_column_mapper)
    pitcher_stats_df = pitcher_stats_df.query('person_id != 0')
    pitcher_stats_df = pitcher_stats_df.drop(columns=['namefield', 'name', 'note'])
    return pitcher_stats_df


def get_boxscore_game_batters_stats(boxscore_obj):
    team_info = get_boxscore_teams(boxscore_obj)

    home_batter_stats_df = get_boxscore_team_batter_stats(boxscore_obj['homeBatters'])
    away_batter_stats_df = get_boxscore_team_batter_stats(boxscore_obj['awayBatters'])

    home_batter_stats_df['team_id'] = team_info['home_team_id']
    away_batter_stats_df['team_id'] = team_info['away_team_id']

    if list(home_batter_stats_df.columns) != list(away_batter_stats_df.columns):
        away_batter_stats_df = away_batter_stats_df[list(home_batter_stats_df.columns)]

    batter_stats_df = pd.concat([home_batter_stats_df, away_batter_stats_df], ignore_index=True)  
    return batter_stats_df 


def get_boxscore_team_batter_stats(batters_list):
    batters_column_mapper = {
        'personId': 'person_id',
        'ab': 'at_bats',
        'r': 'runs',
        'h': 'hits',
        'hr': 'home_runs',
        'rbi': 'runs_batted_in',
        'sb': 'stolen_bases',
        'bb': 'walks',
        'k': 'strikeouts',
        'lob': 'left_on_base',
        'avg': 'batting_average',
        'ops': 'onbase_plus_slugging',
        'obp': 'onbase_percentage',
        'slg': 'slugging_percentage',
        'battingOrder': 'batting_order'
    }

    batters_df = pd.DataFrame(batters_list)
    batters_df = batters_df.rename(columns=batters_column_mapper)
    batters_df = batters_df.query('person_id != 0')
    batters_df = batters_df.drop(columns=['namefield', 'note', 'name', 'position'])
    return batters_df  


def get_boxscore_teams(boxscore_obj):
    teams = boxscore_obj['teamInfo']
    return {'away_team_id': teams['away']['id'], 'home_team_id': teams['home']['id']}


In [52]:
all_batter_stats_df = pd.DataFrame()
all_pitcher_stats_df = pd.DataFrame()

# TO DO: Make this incremental
for game_id in list(sched_df['game_id'])[0:5]:
    batter_stats_df, pitcher_stats_df = get_game_boxscore_stats(game_id)
    batter_stats_df['game_id'] = game_id
    pitcher_stats_df['game_id'] = game_id
    all_batter_stats_df = pd.concat([all_batter_stats_df, batter_stats_df], ignore_index=True)
    all_pitcher_stats_df = pd.concat([all_pitcher_stats_df, pitcher_stats_df], ignore_index=True)


all_batter_stats_df.columns = camelcase_to_snakecase(all_batter_stats_df.columns)
all_pitcher_stats_df.columns = camelcase_to_snakecase(all_pitcher_stats_df.columns)

all_batter_stats_df.to_csv('./test_output/boxscore_player_batter_stats.csv', index=False)
all_pitcher_stats_df.to_csv('./test_output/boxscore_player_pitcher_stats.csv', index=False)

<h3>Seasons API</h3>

In [38]:
seasons = statsapi.get('seasons', params={'ver': 'v1', 'sportId': 1, 'all': True})
seasons_df = pd.DataFrame(seasons['seasons'])
seasons_df.columns = camelcase_to_snakecase(seasons_df.columns)
seasons_df.to_csv('./test_output/seasons.csv', index=False)

<h3>Team API</h3>

In [39]:
def extract_field_from_json(json_obj, field_name):
    return None if field_name not in json_obj else json_obj[field_name]


def parse_json_field(json_obj, json_keys):
    # If no dictionary provide in row, return None values
    if pd.isna(json_obj):
        return pd.Series([None]*len(json_keys))
    
    value_list = []
    for json_key in json_keys:
        value_list.append(extract_field_from_json(json_obj, json_key)) 

    return pd.Series(value_list)


# Retrieve team data from API
teams = statsapi.get('teams', params={'ver': 'v1'})['teams']
teams_df = pd.DataFrame(teams)

# Create list of fields that contain dictionaries as values in the form {'id', ... , 'name': ...}
# to extract data from
dict_fields = ['venue', 'league', 'division', 'sport', 'springLeague', 'springVenue']
for field in dict_fields:
    id_field_name = f'{field.lower()}_id'
    name_field_name = f'{field.lower()}_name'
    teams_df[[id_field_name, name_field_name]] = teams_df[field].apply(parse_json_field, json_keys=['id', 'name'])

    teams_df.drop(columns=field, inplace=True)

# Only extract MLB teams
# Sport ID = 1 is the MLB
mlb_teams_df = teams_df[teams_df['sport_id'] == 1]
mlb_teams_df.columns = camelcase_to_snakecase(mlb_teams_df.columns)

mlb_teams_df.to_csv('./test_output/teams.csv', index=False)

<h3>Leagues API</h3>

In [74]:
league_columns = [
    'league_id', 'name', 'abbreviation', 'active', 'has_wild_card', 'has_split_season', 'num_games', 'has_playoff_points','num_teams', 'num_wildcard_teams'
]

leagues = statsapi.get('league', params={'ver': 'v1', 'sportId': 1})['leagues']
leagues_df = pd.DataFrame(leagues)
leagues_df = leagues_df.rename(columns={'id': 'league_id'})
leagues_df.columns = camelcase_to_snakecase(leagues_df.columns)
leagues_df = leagues_df[league_columns]
leagues_df.to_csv('./test_output/leagues.csv', index=False)

<h3>Divisions API</h3>

In [87]:
divisions_columns = [
    'division_id', 'name', 'name_short', 'abbreviation', 'active', 'has_wildcard', 'num_playoff_teams'
]

divisions = statsapi.get('divisions', params={'ver': 'v1', 'sportId': 1})['divisions']
divisions_df = pd.DataFrame(divisions)
divisions_df = divisions_df.rename(columns={'id': 'division_id'})
divisions_df['league_id'] = divisions_df['league'].apply(parse_json_field, json_keys=['id'])
divisions_df.columns = camelcase_to_snakecase(divisions_df.columns)
divisions_df = divisions_df[divisions_columns] 

divisions_df.to_csv('./test_output/divisions.csv', index=False)


<h3>Team Rosters API</h3>

In [40]:
all_team_rosters_df = pd.DataFrame()

historical_team_rosters_df = pd.DataFrame()
team_roster_fields = ['person_id', 'team_id', 'season_id']

for team_id in mlb_teams_df['id'].unique():   
    for season in [2020, 2021, 2022, 2023, 2024]:
        team_roster = statsapi.get('team_roster', params={'ver': 'v1', 'teamId': team_id, 'season': season})['roster']
        team_roster_df = pd.DataFrame(team_roster)

        team_roster_df['person_id'] = team_roster_df['person'].apply(lambda x: x['id'])
        team_roster_df['team_id'] = team_id
        team_roster_df['season_id'] = season
        team_roster_df = team_roster_df[team_roster_fields]        
        
        historical_team_rosters_df = pd.concat([historical_team_rosters_df, team_roster_df], ignore_index=True)

historical_team_rosters_df.columns = camelcase_to_snakecase(historical_team_rosters_df.columns)

historical_team_rosters_df.to_csv('./test_output/team_rosters.csv', index=False)

<h3>Players API</h3>

In [50]:
players_columns = [
    'player_id', 'first_name', 'last_name', 'primary_number', 'birth_date', 'current_age', 'birth_city', 
    'birth_country', 'height', 'weight', 'active', 'gender', 'mlb_debut_date',  'draft_year', 
    'birth_state_province', 'position_name', 'position_type','bat_side', 'strike_zone_top', 'strike_zone_bottom', 'pitch_hand'
]

players = statsapi.get('sports_players', params={'ver': 'v1', 'sportId': 1, 'season': 2024})['people']
players_df = pd.DataFrame(players)
players_df = players_df.rename(columns={'id': 'player_id'})
players_df[['position_name', 'position_type']] = players_df['primaryPosition'].apply(parse_json_field, json_keys=['name', 'type'])
players_df['bat_side'] = players_df['batSide'].apply(parse_json_field, json_keys=['description'])
players_df['pitch_hand'] = players_df['pitchHand'].apply(parse_json_field, json_keys=['description'])
players_df.columns = camelcase_to_snakecase(players_df.columns)
players_df = players_df[players_columns]

players_df.to_csv('./test_output/players.csv', index=False)
